In [1]:
# Restart kernel and reimport everything
import importlib
import sys

# Clear module cache for wing model
modules_to_clear = [name for name in sys.modules.keys() if 'wing_model' in name]
for module in modules_to_clear:
    del sys.modules[module]
    
print(f"Cleared {len(modules_to_clear)} wing model modules from cache")

Cleared 0 wing model modules from cache


# Unified Volatility Model Calibration

This notebook demonstrates the **unified volatility calibrator framework** for comparing different volatility models using a single, consistent calibration approach.

## Key Features
- **UnifiedVolatilityCalibrator**: Single calibrator that works with any volatility model
- **Multiple Models**: Compare Time-Adjusted Wing Model vs Traditional Wing Model  
- **Visual Comparison**: Interactive plots showing model performance
- **Performance Metrics**: RMSE comparison and parameter analysis

## Architecture Benefits
- Mathematically consistent optimization across all models
- Easy extensibility for new volatility models
- Unified parameter bounds and constraint handling
- Backward compatibility with existing calibrators

## 1. Setup and Configuration

In [2]:
# Import required libraries
import sys
import os
import plotly.graph_objects as go
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Project setup
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.append(project_root)

print(f"📁 Current directory: {current_dir}")
print(f"📁 Project root: {project_root}")
print("✅ Environment setup complete")

📁 Current directory: /home/user/Python/Baseoffset-Fitting-Manager/notebooks
📁 Project root: /home/user/Python/Baseoffset-Fitting-Manager
✅ Environment setup complete


In [3]:
# Create configuration dictionary
config = {
    'date': '20240229',
    'option_expiry': '8MAR24',#'15MAR24',
    'snapshot_time': '2024-02-29T20:12:00',
    'snapshot_tolerance_seconds': 30    
}

print("📋 Configuration created:")
print(f"  - Expiry: {config['option_expiry']}")
print(f"  - Date: {config['date']}")
print(f"  - Snapshot time: {config['snapshot_time']}")

📋 Configuration created:
  - Expiry: 8MAR24
  - Date: 20240229
  - Snapshot time: 2024-02-29T20:12:00


## 2. Data Loading and Preparation

In [4]:
# Load market data
from utils.volatility_fitter.processed_data_loader import create_snapshot_option_chain, load_baseoffset_results, load_option_market_data
from utils.volatility_fitter.volatility_calculator import get_option_chains

date_str = config['date']
snapshot_time = datetime.strptime(config['snapshot_time'], "%Y-%m-%dT%H:%M:%S")
df_baseoffset = load_baseoffset_results(date_str)
df_option_md = load_option_market_data(date_str)

df_snapshot_md = create_snapshot_option_chain(df_option_md, df_baseoffset, snapshot_time)
my_expiry = config['option_expiry']

df_option_chain = get_option_chains(df_snapshot_md, my_expiry, snapshot_time)

print(f"📊 Loaded {len(df_option_chain)} option contracts")
print(f"📅 Expiry: {my_expiry} Time: {snapshot_time}")
df_option_chain.head()

Available expiries: ['26APR24', '2MAR24', '27SEP24', '15MAR24', '31MAY24', '27DEC24', '8MAR24', '22MAR24', '29FEB24', '29MAR24', '1MAR24', '3MAR24', '28JUN24']
📊 Loaded 41 option contracts
📅 Expiry: 8MAR24 Time: 2024-02-29 20:12:00


timestamp,bid_price,ask_price,strike,bid_size,ask_size,bid_price_P,ask_price_P,S,bid_price_fut,ask_price_fut,expiry,tau,bid_size_P,ask_size_P,future_basis
datetime[ns],f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64
2024-02-29 20:12:00,0.312,0.3385,42000,6.4,5.0,0.0003,0.0005,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,7.5,7.9,304.18
2024-02-29 20:12:00,0.2805,0.307,44000,5.0,5.0,0.0005,0.0007,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,0.2,18.2,304.18
2024-02-29 20:12:00,0.2645,0.291,45000,5.0,5.0,0.0005,0.0008,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,3.4,17.4,304.18
2024-02-29 20:12:00,0.2485,0.2635,46000,5.0,6.3,0.0006,0.0009,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,3.1,8.3,304.18
2024-02-29 20:12:00,0.2335,0.2585,47000,5.0,5.0,0.0008,0.0011,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,4.0,12.2,304.18


In [5]:
from utils.pricer.option_constraints import tighten_option_spread
from utils.reporting.html_table_generator import generate_price_comparison_table
from IPython.display import HTML, display

tightened_option_chain = tighten_option_spread(df_option_chain)
print(f"Comparison of old bid/ask proces and the tightened bid/ask price")

# Generate HTML table using external module
display(HTML(generate_price_comparison_table(tightened_option_chain, table_width="70%", font_size="10px")))

Comparison of old bid/ask proces and the tightened bid/ask price


In [6]:
# Get forward price and time to expiry
from utils.volatility_fitter.volatility_calculator import process_option_chain_with_volatilities, process_volatility_with_greeks

df_option_with_vola = process_option_chain_with_volatilities(tightened_option_chain, interest_rate=0.15)
df_option_with_vola_and_greeks = process_volatility_with_greeks(df_option_with_vola)

assert len(df_option_with_vola['F'].unique().to_list()) == 1
assert len(df_option_with_vola['tau'].unique().to_list()) == 1
forward_price = df_option_with_vola['F'][0]
time_to_expiry = df_option_with_vola['tau'][0]

print(f"💰 Forward price: ${forward_price:,.2f}")
print(f"⏰ Time to expiry: {time_to_expiry:.4f} years")


print("✅ DataFrame with vega calculations created successfully using centralized module!")
df_option_with_vola_and_greeks.select(['strike', 'bidVola', 'midVola', 'askVola', 'vega'])

💰 Forward price: $62,229.04
⏰ Time to expiry: 0.0205 years
✅ DataFrame with vega calculations created successfully using centralized module!


strike,bidVola,midVola,askVola,vega
i64,f64,f64,f64,f64
42000,111.67,115.28,118.9,1.71
44000,106.23,108.71,111.19,2.5
45000,100.07,103.44,106.82,2.75
46000,96.39,99.3,102.22,3.19
47000,94.16,96.53,98.9,3.92
…,…,…,…,…
68000,72.81,73.41,74.01,25.94
70000,73.78,75.28,76.77,20.71
72000,76.38,77.34,78.31,16.01


In [7]:
fig = go.Figure()
# Plot with error bars for bid/ask implied volatility
fig.add_trace(go.Scatter(
    x=df_option_with_vola_and_greeks['strike'],
    y=(df_option_with_vola_and_greeks['bidVola_C']+df_option_with_vola_and_greeks['askVola_C'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_C'] - df_option_with_vola_and_greeks['bidVola_C']).abs()/2,
        visible=True, color='blue'),
    mode='markers',
    name='Call IV (Bid/Ask Error Bar)',
    marker=dict(color='blue', symbol='circle'),
    opacity=0.8
))
fig.add_trace(go.Scatter(
    x=df_option_with_vola_and_greeks['strike'],
    y=(df_option_with_vola_and_greeks['bidVola_P']+df_option_with_vola_and_greeks['askVola_P'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_P'] - df_option_with_vola_and_greeks['bidVola_P']).abs()/2,
        visible=True, color='orange'),
    mode='markers',
    name='Put IV (Bid/Ask Error Bar)',
    marker=dict(color='orange', symbol='circle'),
    opacity=0.8
))

fig.add_vline(x=df_option_with_vola_and_greeks['S'][0], line=dict(color='green', dash='dash'), name='Spot Price (S)')

fig.update_layout(
    title=f"{snapshot_time.strftime('%Y-%m-%d %H:%M')}: IV on DERIBIT BTC option for Expiry {my_expiry}",
    xaxis_title="Strike Price",
    yaxis_title="Implied Volatility (%)",
    legend_title="Legend",
    template="plotly_white"
)
fig.show()

## 3. Unified Volatility Model Calibration

Compare different volatility models using the unified calibrator framework:

In [8]:
# Import unified calibrator and both models
from utils.volatility_fitter.unified_volatility_calibrator import UnifiedVolatilityCalibrator
from utils.volatility_fitter.time_adjusted_wing_model.time_adjusted_wing_model import TimeAdjustedWingModel
from utils.volatility_fitter.wing_model.wing_model import WingModel
from utils.volatility_fitter.wing_model.wing_model_parameters import WingModelParameters

# Extract required data from the processed dataframe
strikes_list = df_option_with_vola_and_greeks['strike'].to_list()
market_vols = (df_option_with_vola_and_greeks['midVola']/100).to_list()  # Convert from percentage
market_vegas = df_option_with_vola_and_greeks['vega'].to_list()

print("🚀 Unified Volatility Model Calibration")
print("=" * 50)
print(f"Market Data: {len(strikes_list)} strikes")
print(f"Forward Price: {forward_price:.2f}")
print(f"Time to Expiry: {time_to_expiry:.4f} years")
print(f"Strike range: ${min(strikes_list):,.0f} - ${max(strikes_list):,.0f}")

# Dictionary to store results for comparison
model_results = {}

🚀 Unified Volatility Model Calibration
Market Data: 41 strikes
Forward Price: 62229.04
Time to Expiry: 0.0205 years
Strike range: $42,000 - $76,000


In [9]:
# 🔧 Load Volatility Configuration
from config.config_loader import load_volatility_config

print("⚙️ Loading Volatility Configuration...")
vol_config = load_volatility_config()

print(f"✅ Configuration loaded: {vol_config.config_type}")
print(f"📁 Config file: {vol_config.config_path.split('/')[-1]}")
print(f"📅 Date: {vol_config.date_str}")
print(f"🎯 Max RMSE threshold: {vol_config.max_rmse_threshold}")
print(f"🔧 Calibration method: {vol_config.calibration_method}")
print(f"⚙️ Tolerance: {vol_config.calibration_tolerance}")
print(f"🔄 Max iterations: {vol_config.max_calibration_iterations}")

# Show available model configurations
wing_config = vol_config.get_model_config('wing_model')
ta_config = vol_config.get_model_config('time_adjusted_wing_model')

print(f"\n🏗️ Available Model Configurations:")
print(f"   📈 Wing Model: {'✅ CONFIGURED' if wing_config else '❌ NOT FOUND'}")
print(f"   ⏰ Time-Adjusted Wing Model: {'✅ CONFIGURED' if ta_config else '❌ NOT FOUND'}")

if wing_config and 'initial_params' in wing_config:
    wing_params = vol_config.get_initial_params('wing_model')
    print(f"   📊 Wing Model initial vr (ATM vol): {wing_params.get('vr', 'N/A')}")

print("🎉 Configuration ready for volatility calibration!")

⚙️ Loading Volatility Configuration...
✅ Configuration loaded: volatility
📁 Config file: volatility_config.yaml
📅 Date: 20240229
🎯 Max RMSE threshold: 0.1
🔧 Calibration method: SLSQP
⚙️ Tolerance: 1e-10
🔄 Max iterations: 1000

🏗️ Available Model Configurations:
   📈 Wing Model: ✅ CONFIGURED
   ⏰ Time-Adjusted Wing Model: ✅ CONFIGURED
   📊 Wing Model initial vr (ATM vol): 0.7
🎉 Configuration ready for volatility calibration!


### 3.1 Time-Adjusted Wing Model Calibration

In [19]:
# 1️⃣ Time-Adjusted Wing Model Calibration with Configuration
from utils.volatility_fitter.time_adjusted_wing_model import create_time_adjusted_wing_model_from_result
import numpy as np

print("🔧 Calibrating Time-Adjusted Wing Model...")
print("   Using configuration-driven parameters and settings")

# Get initial parameters from configuration
config_initial_params = vol_config.get_initial_params('time_adjusted_wing_model')
print(f"📋 Config initial parameters: {config_initial_params}")

# Convert config parameters to initial guess array
# [atm_vol(vr), slope(sr), call_curve(cc), put_curve(pc), up_cutoff(uc), down_cutoff(dc), up_smoothing(usm), down_smoothing(dsm)]
initial_guess = list(config_initial_params.values())

# Create initial parameters using the helper function (creates WingModelParameters with proper mapping)
ta_initial_params = create_time_adjusted_wing_model_from_result(
    result=initial_guess,
    forward_price=forward_price,
    ref_price=forward_price,  # Use same as forward price
    time_to_expiry=time_to_expiry
)

# Get calibrator settings from configuration
calibration_config = vol_config.get_calibration_config()
unified_config = calibration_config.get('unified_calibrator', {})

# Create unified calibrator using configuration settings
ta_calibrator = UnifiedVolatilityCalibrator(
    model_class=TimeAdjustedWingModel,
    enable_bounds=unified_config.get('enable_bounds', True), 
    tolerance=vol_config.calibration_tolerance,
    method=vol_config.calibration_method,
    max_iterations=vol_config.max_calibration_iterations,
    arbitrage_penalty=unified_config.get('arbitrage_penalty', 1e5)
)

print(f"🔧 Calibrator settings from config:")
print(f"   Method: {vol_config.calibration_method}")
print(f"   Tolerance: {vol_config.calibration_tolerance}")
print(f"   Max iterations: {vol_config.max_calibration_iterations}")

# Get parameter bounds from configuration using the corrected method
ta_bounds = vol_config.get_parameter_bounds('time_adjusted_wing_model')
print(f"📏 Parameter bounds from config: {len(ta_bounds)} parameters configured")
print(f"📏 Bounds: {ta_bounds}")

# Ensure data is in correct format (convert to numpy arrays for safety)
strikes_array = np.array(strikes_list, dtype=float)
vols_array = np.array(market_vols, dtype=float)  
vegas_array = np.array(market_vegas, dtype=float)

print(f"🔢 Data arrays converted: strikes={len(strikes_array)}, vols={len(vols_array)}, vegas={len(vegas_array)}")

# Perform calibration
try:
    ta_result = ta_calibrator.calibrate(
        initial_params=ta_initial_params,
        strikes=strikes_array,
        market_volatilities=vols_array,
        market_vegas=vegas_array,
        parameter_bounds=ta_bounds,
        enforce_arbitrage_free=True
    )
    
    # Store results
    model_results['Time-Adjusted Wing'] = {
        'result': ta_result,
        'calibrator': ta_calibrator,
        'model_class': TimeAdjustedWingModel
    }

    if ta_result.success:
        print(f"✅ Time-Adjusted Wing Model calibration: SUCCESS")
        print(f"   Final RMSE: {ta_result.error:.6f}")
        
        # Check against configuration threshold
        if ta_result.error <= vol_config.max_rmse_threshold:
            print(f"   🎯 RMSE below config threshold ({vol_config.max_rmse_threshold:.3f}): ✅ PASS")
        else:
            print(f"   ⚠️ RMSE above config threshold ({vol_config.max_rmse_threshold:.3f}): REVIEW NEEDED")
        
        print(f"   Optimized parameters:")
        
        # Show parameter values using the updated mapping  
        param_names = ta_result.parameters.get_parameter_names()
        param_values = ta_result.parameters.get_fitted_vol_parameter()
        for name, value in zip(param_names, param_values):
            print(f"     {name}: {value:.6f}")
    else:
        print(f"❌ Time-Adjusted Wing Model calibration: FAILED")
        print(f"   Error: {ta_result.message}")

except Exception as e:
    print(f"🚨 EXCEPTION during calibration: {e}")
    import traceback
    traceback.print_exc()

🔧 Calibrating Time-Adjusted Wing Model...
   Using configuration-driven parameters and settings
📋 Config initial parameters: {'vr': 0.7, 'sr': 0.01, 'cc': 1.0, 'pc': 1.0, 'uc': 0.2, 'dc': -0.2, 'usm': 0.5, 'dsm': 0.5}
🔧 Calibrator settings from config:
   Method: SLSQP
   Tolerance: 1e-10
   Max iterations: 1000
📏 Parameter bounds from config: 8 parameters configured
📏 Bounds: [(0.05, 5.0), (-2.0, 2.0), (-5.0, 5.0), (-5.0, 5.0), (-2.0, 0.0), (0.0, 2.0), (0.01, 2.0), (0.01, 2.0)]
🔢 Data arrays converted: strikes=41, vols=41, vegas=41
❌ Time-Adjusted Wing Model calibration: FAILED
   Error: Calibration error: can't multiply sequence by non-int of type 'float'


In [25]:
# 2️⃣ Traditional Wing Model Calibration with Configuration
print("🔧 Calibrating Traditional Wing Model...")
print("   Using configuration-driven parameters and settings")

# Get initial parameters from configuration for wing model
config_wing_params = vol_config.get_initial_params('wing_model')
print(f"📋 Config wing parameters: {config_wing_params}")

# Create initial parameters using configuration values
wing_initial_params = WingModelParameters(
    forward_price=forward_price,
    ref_price=forward_price,
    time_to_expiry=time_to_expiry,
    vr=config_wing_params.get('vr', 0.7),        # volatility reference  
    sr=config_wing_params.get('sr', 0.01),       # slope reference
    pc=config_wing_params.get('pc', 1.0),        # put curvature
    cc=config_wing_params.get('cc', 1.0),        # call curvature
    dc=config_wing_params.get('dc', -0.2),       # down cutoff (LOG MONEYNESS) 
    uc=config_wing_params.get('uc', 0.2),        # up cutoff (LOG MONEYNESS)
    dsm=config_wing_params.get('dsm', 0.5),      # down smoothing
    usm=config_wing_params.get('usm', 0.5)       # up smoothing
)

print(f"✅ Wing initial parameters: vr={wing_initial_params.vr:.3f}, sr={wing_initial_params.sr:.3f}")
print(f"   dc={wing_initial_params.dc:.3f}, uc={wing_initial_params.uc:.3f}")

# Get parameter bounds from configuration
wing_bounds = vol_config.get_parameter_bounds('wing_model')

# Create calibrator using configuration settings
wing_calibrator = UnifiedVolatilityCalibrator(
    model_class=WingModel,
    enable_bounds=unified_config.get('enable_bounds', True),
    tolerance=vol_config.calibration_tolerance,
    method=vol_config.calibration_method,
    arbitrage_penalty=unified_config.get('arbitrage_penalty', 1e5),
    max_iterations=vol_config.max_calibration_iterations
)

print(f"🔧 Wing calibrator settings (same as TA model from config):")
print(f"   Method: {vol_config.calibration_method}")
print(f"   Tolerance: {vol_config.calibration_tolerance}")

# Calibrate
wing_result = wing_calibrator.calibrate(
    initial_params=wing_initial_params,
    strikes=strikes_list,
    market_volatilities=market_vols,
    market_vegas=market_vegas,
    parameter_bounds=wing_bounds,
    enforce_arbitrage_free=True
)

# Store results
model_results['Traditional Wing'] = {
    'result': wing_result,
    'calibrator': wing_calibrator,
    'model_class': WingModel
}

if wing_result.success:
    print(f"✅ Traditional Wing Model calibration: SUCCESS")
    print(f"   Final RMSE: {wing_result.error:.6f}")
    
    # Check against configuration threshold
    if wing_result.error <= vol_config.max_rmse_threshold:
        print(f"   🎯 RMSE below config threshold ({vol_config.max_rmse_threshold:.3f}): ✅ PASS")
    else:
        print(f"   ⚠️ RMSE above config threshold ({vol_config.max_rmse_threshold:.3f}): REVIEW NEEDED")
    
    print(f"   Optimized parameters:")
    
    # Show parameter values
    param_names = wing_result.parameters.get_parameter_names()
    param_values = wing_result.parameters.get_fitted_vol_parameter()
    for name, value in zip(param_names, param_values):
        print(f"     {name}: {value:.6f}")
else:
    print(f"❌ Traditional Wing Model calibration: FAILED")
    print(f"   Error: {wing_result.message}")

print()

🔧 Calibrating Traditional Wing Model...
   Using configuration-driven parameters and settings
📋 Config wing parameters: {'vr': 0.7, 'sr': 0.01, 'pc': 1.0, 'cc': 1.0, 'dc': -0.2, 'uc': 0.2, 'dsm': 0.5, 'usm': 0.5}
✅ Wing initial parameters: vr=0.700, sr=0.010
   dc=-0.200, uc=0.200
🔧 Wing calibrator settings (same as TA model from config):
   Method: SLSQP
   Tolerance: 1e-10
❌ Traditional Wing Model calibration: FAILED
   Error: Calibration error: can't multiply sequence by non-int of type 'float'



### 3.2 Model Comparison & Visualization

In [ ]:
# 📊 Model Performance Comparison with Configuration Validation
print("\n📊 Model Performance Summary:")
print("=" * 50)

# Get validation config
validation_config = vol_config.get_validation_config()
quality_config = validation_config.get('quality', {})

print(f"🎯 Configuration thresholds:")
print(f"   Max RMSE: {vol_config.max_rmse_threshold:.3f}")
print(f"   Min R²: {quality_config.get('min_r_squared', 0.8):.3f}")
print()

best_model = None
best_rmse = float('inf')
passing_models = []

for model_name, data in model_results.items():
    result = data['result']
    rmse = result.error
    
    # Check against configuration thresholds
    rmse_pass = rmse <= vol_config.max_rmse_threshold
    status = "✅ PASS" if result.success and rmse_pass else "❌ FAIL"
    
    if result.success and rmse_pass:
        passing_models.append(model_name)
    
    print(f"{model_name:25}: RMSE = {rmse:.6f} {status}")
    
    if result.success and rmse < best_rmse:
        best_rmse = rmse
        best_model = model_name

print(f"\n🎯 Configuration Validation Results:")
print(f"   Models meeting RMSE threshold: {len(passing_models)} / {len(model_results)}")
print(f"   Passing models: {', '.join(passing_models) if passing_models else 'None'}")

if best_model:
    if len(model_results) == 2:
        model_errors = [data['result'].error for data in model_results.values()]
        improvement = abs((model_errors[0] - model_errors[1]) / max(model_errors)) * 100
        print(f"\n🏆 Best Model: {best_model} (RMSE: {best_rmse:.6f})")
        print(f"🔧 Performance difference: {improvement:.2f}%")
        
        if best_rmse <= vol_config.max_rmse_threshold:
            print(f"✅ Best model meets configuration quality threshold!")
        else:
            print(f"⚠️ Best model exceeds configuration RMSE threshold ({vol_config.max_rmse_threshold:.3f})")
else:
    print("\n❌ No successful calibrations found")

In [ ]:
# 📈 Generate Volatility Surface Comparison Plot
import numpy as np

# Create extended strike range for smooth curves
min_strike = min(strikes_list) * 0.9
max_strike = max(strikes_list) * 1.1
extended_strikes = np.linspace(min_strike, max_strike, 100)

fig = go.Figure()

# Extract bid/ask volatilities for error bars
market_bid_vols = (df_option_with_vola_and_greeks['bidVola']/100).to_list()
market_ask_vols = (df_option_with_vola_and_greeks['askVola']/100).to_list()

# Add market data points with bid/ask error bars
fig.add_trace(go.Scatter(
    x=strikes_list, y=market_vols, mode='markers', name='Market IV (Bid/Ask)',
    error_y=dict(type='data', symmetric=False,
        array=[ask - mid for ask, mid in zip(market_ask_vols, market_vols)],  # Upper error
        arrayminus=[mid - bid for bid, mid in zip(market_bid_vols, market_vols)],  # Lower error
        visible=True, color='rgba(0,0,0,0.3)', thickness=2, width=3
    ),
    marker=dict(size=6, color='black', symbol='circle'),
    hovertemplate='Strike: %{x}<br>Mid Vol: %{y:.4f}<br>Bid-Ask Spread<extra></extra>',
    opacity=0.8
))
# Add vertical lines for reference
forward_price = df_option_with_vola_and_greeks['F'][0]    
fig.add_vline(x=forward_price, line=dict(color='purple', dash='dot', width=2), annotation_text=f"Forward: {forward_price:.0f}")

# Define colors for models
model_colors = {'Wing Model': 'blue', 'Time-Adjusted Wing Model': 'red'}

# Add model curves with fitted error bars
for model_name, data in model_results.items():        
    result = data['result']
    model_class = data['model_class']
    color = model_colors.get(model_name, 'green')
    
    # Create model instance with optimized parameters
    model = model_class(result.parameters)
    
    # Calculate model volatilities for extended strikes
    model_vols = []
    for strike in extended_strikes:
        try:
            vol = model.calculate_volatility_from_strike(strike)
            model_vols.append(vol)
        except:
            model_vols.append(np.nan)
    
    # Add model curve
    fig.add_trace(go.Scatter(
        x=extended_strikes, y=model_vols, mode='lines',
        name=f'{model_name} (RMSE: {result.error:.6f})',
        line=dict(color=color, width=2),
        hovertemplate=f'{model_name}<br>Strike: %{{x}}<br>Vol: %{{y:.4f}}<extra></extra>'
    ))    

# Update layout
title = f"{(ts:=df_option_with_vola_and_greeks['timestamp'][0])}: Fitted Model vs Market Data - {my_expiry} Expiry"

fig.update_layout(
    title=dict(
        text=f"{title}<br><span style='font-size:14px'>Forward: {forward_price:.0f} | τ: {time_to_expiry:.4f}",# | RMSE: {fitter_result.error:.6f}<br> {param_string}</span>",
        x=0.5
    ),
    xaxis_title='Strike Price',
    yaxis_title='Implied Volatility',
    width=900,
    height=600,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)'),
    template='plotly_white'
)

fig.show()

## 4. Summary

This notebook demonstrates the unified volatility calibrator framework that provides:

In [ ]:
# 🎯 Final Summary with Configuration Integration
print("🎯 Configuration-Driven Volatility Calibrator Framework Summary")
print("=" * 65)
print()
print("⚙️ Configuration Management:")
print(f"   • Config file: {vol_config.config_path.split('/')[-1]}")
print(f"   • Config type: {vol_config.config_type}")
print(f"   • Date: {vol_config.date_str}")
print(f"   • RMSE threshold: {vol_config.max_rmse_threshold:.3f}")
print(f"   • Calibration method: {vol_config.calibration_method}")
print(f"   • Max iterations: {vol_config.max_calibration_iterations}")
print()
print("✅ Key Benefits:")
print("   • Configuration-driven parameter initialization")
print("   • Centralized calibration settings and validation thresholds")
print("   • Single calibrator works with multiple volatility models")
print("   • Mathematically consistent optimization across all models")
print("   • Easy to add new models by inheriting from BaseVolatilityModel")
print("   • Unified parameter bounds and constraints handling")
print()
print("📊 Model Performance Results:")
for model_name, data in model_results.items():
    result = data['result']
    if result.success:
        rmse_status = "✅ PASS" if result.error <= vol_config.max_rmse_threshold else "⚠️ THRESHOLD"
        print(f"   • {model_name}: RMSE = {result.error:.6f} {rmse_status}")
        
        # Show top 3 parameters
        param_names = result.parameters.get_parameter_names()[:3]
        param_values = result.parameters.get_fitted_vol_parameter()[:3]
        param_str = ", ".join([f"{name}={val:.3f}" for name, val in zip(param_names, param_values)])
        print(f"     Key params: {param_str}...")

if best_model:
    print(f"\n🏆 Best performing model: {best_model} (RMSE: {best_rmse:.6f})")
    
print(f"\n🔧 Configuration Benefits Demonstrated:")
print(f"   • Parameters loaded from volatility_config.yaml")
print(f"   • Calibrator settings centrally managed")
print(f"   • Quality thresholds automatically applied")
print(f"   • Consistent validation across all models")
print(f"   • Easy to modify settings via config file")
print()
print("📈 Visual comparison shows model fit quality against market data")
print("🎉 Configuration-driven volatility fitting complete!")